In [7]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta

In [8]:
# Sample Inputs
investment_amount = 10000  # Total investment amount
start_date = "2024-01-01"  # Investment period start date
end_date = "2024-05-30"  # Investment period end date
end_date2 = "2024-05-29"  # Investment period end date

In [9]:
stocks = [
	{"Ticker": "ADANIPORTS.NS", "Weightage": 50.0000},
	{"Ticker": "APOLLOHOSP.NS", "Weightage": 25.0000},
	{"Ticker": "ASIANPAINT.NS", "Weightage": 25.0000},
	# {"Ticker": "AXISBANK.NS", "Weightage": 0.0257},
	# {"Ticker": "BAJAJFINSV.NS", "Weightage": 0.012},
	# {"Ticker": "BAJFINANCE.NS", "Weightage": 0.0237},
	# {"Ticker": "BHARTIARTL.NS", "Weightage": 0.0233},
	# {"Ticker": "BPCL.NS", "Weightage": 0.0046},
	# {"Ticker": "BRITANNIA.NS", "Weightage": 0.0052},
	# {"Ticker": "CIPLA.NS", "Weightage": 0.0068},
	# {"Ticker": "COALINDIA.NS", "Weightage": 0.0051},
	# {"Ticker": "DIVISLAB.NS", "Weightage": 0.0077},
	# {"Ticker": "DRREDDY.NS", "Weightage": 0.0067},
	# {"Ticker": "EICHERMOT.NS", "Weightage": 0.0049},
	# {"Ticker": "GRASIM.NS", "Weightage": 0.0085},
	# {"Ticker": "HCLTECH.NS", "Weightage": 0.0153},
	# {"Ticker": "HDFCBANK.NS", "Weightage": 0.081},
	# {"Ticker": "HDFCLIFE.NS", "Weightage": 0.0072},
	# {"Ticker": "HEROMOTOCO.NS", "Weightage": 0.0043},
	# {"Ticker": "HINDALCO.NS", "Weightage": 0.0094},
	# {"Ticker": "HINDUNILVR.NS", "Weightage": 0.0267},
	# {"Ticker": "ICICIBANK.NS", "Weightage": 0.069},
	# {"Ticker": "INDUSINDBK.NS", "Weightage": 0.0085},
	# {"Ticker": "INFY.NS", "Weightage": 0.0766},
	# {"Ticker": "ITC.NS", "Weightage": 0.0303},
	# {"Ticker": "JSWSTEEL.NS", "Weightage": 0.0094},
	# {"Ticker": "KOTAKBANK.NS", "Weightage": 0.0351},
	# {"Ticker": "LT.NS", "Weightage": 0.0274},
	# {"Ticker": "MARUTI.NS", "Weightage": 0.0137},
	# {"Ticker": "NESTLEIND.NS", "Weightage": 0.0087},
	# {"Ticker": "NTPC.NS", "Weightage": 0.0099},
	# {"Ticker": "ONGC.NS", "Weightage": 0.0078},
	# {"Ticker": "POWERGRID.NS", "Weightage": 0.0104},
	# {"Ticker": "RELIANCE.NS", "Weightage": 0.1286},
	# {"Ticker": "SBILIFE.NS", "Weightage": 0.0065},
	# {"Ticker": "SBIN.NS", "Weightage": 0.0254},
	# {"Ticker": "SHREECEM.NS", "Weightage": 0.0046},
	# {"Ticker": "SUNPHARMA.NS", "Weightage": 0.0134},
	# {"Ticker": "TATACONSUM.NS", "Weightage": 0.0066},
	# {"Ticker": "TATAMOTORS.NS", "Weightage": 0.0105},
	# {"Ticker": "TATASTEEL.NS", "Weightage": 0.0137},
	# {"Ticker": "TCS.NS", "Weightage": 0.0491},
	# {"Ticker": "TECHM.NS", "Weightage": 0.0105},
	# {"Ticker": "TITAN.NS", "Weightage": 0.0137},
	# {"Ticker": "ULTRACEMCO.NS", "Weightage": 0.0102},
	# {"Ticker": "UPL.NS", "Weightage": 0.006},
	# {"Ticker": "WIPRO.NS", "Weightage": 0.0101},
]

In [10]:
def calculate_rsi(data, periods=14):
	"""
	Calculates the Relative Strength Index (RSI) for a given DataFrame.

	Args:
		data: pandas DataFrame containing the 'Close' price data.
		periods: Number of periods for RSI calculation (default: 14).

	Returns:
		pandas Series containing the RSI values.
	"""

	delta = data["Close"].diff()
	gain = (delta.where(delta > 0, 0)).fillna(0)
	loss = (delta.where(delta < 0, 0)).fillna(0)

	avg_gain = gain.rolling(window=periods).mean()
	avg_loss = abs(loss.rolling(window=periods).mean())

	rs = avg_gain / avg_loss
	rsi = 100 - (100 / (1 + rs))

	return rsi

In [11]:
# Ensure start_date and end_date are timestamps
start_date = pd.Timestamp(start_date)
end_date = pd.Timestamp(end_date)

results = []

for stock in stocks:
    ticker = stock["Ticker"]
    weightage = stock["Weightage"]

    data = yf.download(ticker, start=start_date, end=end_date)

    if data.empty:
        print(f"Data for {ticker} is not available. Skipping.")
        continue

    # data["RSI"] = ta.rsi(data["Close"])
    data["RSI"] = calculate_rsi(data)

    print(len(data["Close"]))

    # Print the results
    print("Adj Close:")
    print(data["Close"].head()) 

    print("RSI:")
    print(data["RSI"].head()) 

    data["Adjusted Weightage"] = weightage

    data.loc[data["RSI"] > 90, "Adjusted Weightage"] -= 0.003
    data.loc[
		(data["RSI"] > 80) & (data["RSI"] <= 90), "Adjusted Weightage"
	] -= 0.002
    data.loc[
		(data["RSI"] > 70) & (data["RSI"] <= 80), "Adjusted Weightage"
	] -= 0.001
    data.loc[data["RSI"] < 20, "Adjusted Weightage"] += 0.003
    data.loc[
		(data["RSI"] >= 20) & (data["RSI"] < 30), "Adjusted Weightage"
	] += 0.002
    data.loc[
		(data["RSI"] >= 30) & (data["RSI"] <= 70), "Adjusted Weightage"
	] += 0.001

    data = data.fillna({"Adjusted Weightage": 0, "Adj Close": 1})   
    data.to_csv("data.csv")

    filtered_data = data.loc[ data.index == end_date2]
    if filtered_data.empty:
        print(
			f"No data for {ticker} on {start_date.date()} or {end_date.date()}. Skipping."
		)
        continue

    filtered_data.to_csv("test.csv")

    for index, row in filtered_data.iterrows():
        adjusted_weightage_value = row["Adjusted Weightage"]

        if isinstance(row["Adjusted Weightage"], pd.Series):
            adjusted_weightage_value = row["Adjusted Weightage"].iloc[0]

        print(adjusted_weightage_value)
        print(type(adjusted_weightage_value))

        # Append results
        results.append(
            {
                "Ticker": ticker,
                "Date": index.date(),
                "Weightage": weightage,
                "Adjusted Weightage": row["Adjusted Weightage"].iloc[0],
                "Open": row["Open"].iloc[0],
                "Close": row["Close"].iloc[0],
                "RSI": row["RSI"].iloc[0],
            }
        )
# Convert results to DataFrame
summary = pd.DataFrame(results)

# Debugging: Check the actual column names in summary
print("Column names in summary:", summary.columns)

# Reorder columns to match the output format
expected_columns = [
	"Ticker",
	"Date",
	"Weightage",
	"Adjusted Weightage",
	"Open",
	"Close",
	"RSI",
]
missing_columns = [col for col in expected_columns if col not in summary.columns]

if missing_columns:
	print(f"Missing columns: {missing_columns}")
else:
	# Reorder columns if all are present
	summary = summary[expected_columns]

# Display the summary
print("\nInvestment Summary:")
print(summary)

# Optional: Save to CSV
summary.to_csv("investment_summary.csv", index=False)

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


99
Adj Close:
Ticker      ADANIPORTS.NS
Date                     
2024-01-01    1047.849976
2024-01-02    1078.400024
2024-01-03    1094.250000
2024-01-04    1123.199951
2024-01-05    1154.250000
RSI:
Date
2024-01-01   NaN
2024-01-02   NaN
2024-01-03   NaN
2024-01-04   NaN
2024-01-05   NaN
Name: RSI, dtype: float64
50.001
<class 'numpy.float64'>
99
Adj Close:
Ticker      APOLLOHOSP.NS
Date                     
2024-01-01    5750.049805
2024-01-02    5746.350098
2024-01-03    5768.299805
2024-01-04    5762.200195
2024-01-05    5754.250000
RSI:
Date
2024-01-01   NaN
2024-01-02   NaN
2024-01-03   NaN
2024-01-04   NaN
2024-01-05   NaN
Name: RSI, dtype: float64
25.001
<class 'numpy.float64'>
99
Adj Close:
Ticker      ASIANPAINT.NS
Date                     
2024-01-01    3396.100098
2024-01-02    3391.350098
2024-01-03    3373.600098
2024-01-04    3379.949951
2024-01-05    3355.550049
RSI:
Date
2024-01-01   NaN
2024-01-02   NaN
2024-01-03   NaN
2024-01-04   NaN
2024-01-05   NaN
Name: RSI, dt

[*********************100%***********************]  1 of 1 completed


99
Adj Close:
Ticker      AXISBANK.NS
Date                   
2024-01-01  1097.699951
2024-01-02  1093.250000
2024-01-03  1098.599976
2024-01-04  1123.400024
2024-01-05  1136.949951
RSI:
Date
2024-01-01   NaN
2024-01-02   NaN
2024-01-03   NaN
2024-01-04   NaN
2024-01-05   NaN
Name: RSI, dtype: float64
0.0267
<class 'numpy.float64'>


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

99
Adj Close:
Ticker      BAJAJFINSV.NS
Date                     
2024-01-01    1676.400024
2024-01-02    1686.199951
2024-01-03    1677.650024
2024-01-04    1701.949951
2024-01-05    1709.500000
RSI:
Date
2024-01-01   NaN
2024-01-02   NaN
2024-01-03   NaN
2024-01-04   NaN
2024-01-05   NaN
Name: RSI, dtype: float64
0.013000000000000001
<class 'numpy.float64'>


99
Adj Close:
Ticker      BAJFINANCE.NS
Date                     
2024-01-01    7299.049805
2024-01-02    7430.049805
2024-01-03    7384.799805
2024-01-04    7705.549805
2024-01-05    7711.149902
RSI:
Date
2024-01-01   NaN
2024-01-02   NaN
2024-01-03   NaN
2024-01-04   NaN
2024-01-05   NaN
Name: RSI, dtype: float64
0.0247
<class 'numpy.float64'>


[*********************100%***********************]  1 of 1 completed

99
Adj Close:
Ticker      BHARTIARTL.NS
Date                     
2024-01-01    1013.049988
2024-01-02    1021.700012
2024-01-03    1035.199951
2024-01-04    1047.900024
2024-01-05    1046.599976
RSI:
Date
2024-01-01   NaN
2024-01-02   NaN
2024-01-03   NaN
2024-01-04   NaN
2024-01-05   NaN
Name: RSI, dtype: float64
0.0223
<class 'numpy.float64'>
Column names in summary: Index(['Ticker', 'Date', 'Weightage', 'Adjusted Weightage', 'Open', 'Close',
       'RSI'],
      dtype='object')

Investment Summary:
          Ticker        Date  Weightage  Adjusted Weightage         Open  \
0  ADANIPORTS.NS  2024-05-29    50.0000             50.0010  1406.000000   
1  APOLLOHOSP.NS  2024-05-29    25.0000             25.0010  5909.899902   
2  ASIANPAINT.NS  2024-05-29    25.0000             25.0010  2896.000000   
3    AXISBANK.NS  2024-05-29     0.0257              0.0267  1172.000000   
4  BAJAJFINSV.NS  2024-05-29     0.0120              0.0130  1594.150024   
5  BAJFINANCE.NS  2024-05-29     0.0